In [ ]:
import sys
import os
import numpy as np 
import matplotlib.pyplot as plt


src_path = os.path.expanduser('~/source/')
sys.path.append(src_path)


In [ ]:
#greg tools
sys.path.append(src_path+'simtools')
sys.path.append(src_path+'simtools/infoenginessims')

from integrators import rkdeterm_eulerstoch
from dynamics import langevin_underdamped
import simulation
from simprocedures import basic_simprocedures as sp
from simprocedures import running_measurements as rp
from simprocedures import trajectory_measurements as tp
#from infoenginessims.api import *

In [1]:
#kyle tools
from sus.protocol_designer import Potential, Protocol, System, Compound_Protocol
from informational_states.measure import MeasurementDevice, Measurement, TrajectoryEnsemble
import kyle_tools as kt

In [ ]:
# SQUID tools
from FQ_sympy_functions import DeviceParams
from bit_flip_sweep import set_systems, check_device

In [ ]:
plt.rcParams['font.size'] = 16
legendsize = 6
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'

rc_dict = {'axes.labelsize':'large', 'ytick.right':False,'legend.loc':'upper right', 'legend.fontsize':'xx-small', 'figure.autolayout':True, 'figure.figsize': (12,9)}
plt.rc('grid', linestyle="-", color='black')

for key,value in rc_dict.items():
    plt.rcParams[key] = value

# These are the physical parameters that characterize the system

In [ ]:
# set up the device here, for example:
Dev= DeviceParams()
print('before change:', Dev.I_minus, Dev.R, Dev.C)

Dev.change_vals({'I_minus':0, 'R':371})
# NOTE: ive made R really small so we can see the damping more quickly.

print('after change:',Dev.I_minus, Dev.R, Dev.C)
check_device(Dev)


#some alternate parameter sets

#params 1:
#kT_prime = .11
#C = 10*400E-12
#R = 371
#L = 10E-10

#params 2:
#kT_prime = 6.9E-24/1.38E-23
#C = 530E-15
#R = 2.1
#L = 140E-12


In [ ]:
#params Dev:
kT = Dev.get_temp()*1.38E-23
C = Dev.C
R = Dev.R
L = Dev.L

#these are some relevant dimensionful scales: alpha is the natural units for the JJ fluxes and U_0 is the natural scale for the potential
alpha = Dev.alpha

#IMPORTANT: all energies are measured in units of U_0
U_0 = alpha**2 / L
h = 6.63E-34

#these are important dimensionless simulation quantities, accounting for 
#m being measured in units of C, lambda in units of 1/R, energy in units of U_0
m_prime = np.array((1, 1/4))
lambda_prime = np.array((2, 1/2))
kT_prime = kT/U_0

print('Some Common Circuit Parameters')
Q = R*np.sqrt(C/L)
print( 'Q:{:.2f}'.format(Q))
frq = 1/(2*np.pi*np.sqrt(L*C))
print('f_LC in GHz:{:.2f}'.format(frq/1E9))
print('ring down in ns:{:.2f}'.format(1E9*Q/frq))
print('ring down in sqrt(LC):{:.2f}'.format( (Q/frq)/np.sqrt(L*C)))
j_c = alpha/L
print('critical current density in mu_A:{:.2f}'.format(1E6*j_c))
print('Energy Scale')
print('U_0/kT:{:.2f}'.format(1/kT_prime))
print('kT/(h*f)',kT_prime*U_0 / (h*frq))




# Here we define the $\varphi_{xdc}$ protocol

In [ ]:
protocol_duration = 5.43
initial_run_length = 15

# d_store comp is how far phi_{xdc} goes on either side of the bifurcation point
store, comp = set_systems(Dev, comp_tau=protocol_duration, d_store_comp=[.2,.2])
#store and comp are two systms, each holding a protocol with external fluxes fixed at the storage and computational potentials

# now, we make the comp protocol turn off the computational protocol at the end
# this way, it goes from computational potential to storage potential
comp.protocol.params[:,1] = store.protocol.params[:,1]

# this means it will turn off suddenly at the end time, like a step function
comp.protocol.interpolation = 'end_step'

# this creates a copy of tyhe computational protocol, but with the order reversed
# basically, this one will go fro the storage potential to the computational potential
initialization = comp.protocol.copy()
initialization.reverse()

#now, we normalize the initialization protocol and stretch the time so its as long as we want
initialization.normalize()
initialization.time_stretch(initial_run_length)

#time shift the computational protocol so it starts at the end of the initialization protocol
comp.protocol.time_shift(initial_run_length)

# now, we overwrite the computational protocol with a Compound_Protocol
# this will do each protocol in the list in order
comp.protocol = Compound_Protocol([initialization, comp.protocol])



In [ ]:
# plots the protocol parameters over time, time is in units of sqrt(LC)
fig, ax = comp.protocol.show_params(which=[1,2], param_labels=['$\\varphi_{x}$', '$\\varphi_{xdc}$'], show=False);
fig.set_size_inches(10, 4)


In [ ]:
#this cell defines what system you want to simulate and how many trials to run.
#generally no need to do lots of trials while prototyping protocols

N = 5_000
system= comp
eq_sys = comp

system.mass= m_prime
eq_sys.mass = m_prime

system.potential.scale=1

#initialize the state in an EQ distribution.
initial_state = eq_sys.eq_state(N, t=0, beta=1/(kT_prime), resolution=1_000, manual_domain=[[-4,-4],[4,0]], axes=[1,2])

# Next few cells are visualization checks that your system is set up how you want

In [ ]:
#this cell checks for closeness to equilibrium by checking the generalized equipartition theorem
#the true equilibrium dist will yield zeros for all elements, but only when when n -> infinity
#this can take a lot of trials to converge

I = np.zeros((4,4))
for i in range(4):
    I[i,i] = 1

ept_test = system.check_local_eq(initial_state, 0)/kT_prime - I 
ept_mean, ept_std = np.mean(ept_test, axis=0), np.std(ept_test, axis=0)

element_labels = [ f'${i+1},{j+1}$' for i in range(4) for j in range(4)]

fig, ax = plt.subplots(figsize=(14,4))


ax.errorbar(element_labels, ept_mean.ravel(), yerr = 3*ept_std.ravel()/np.sqrt(N), fmt='o', capsize=3, capthick=2, elinewidth=2, color='black')
ax.axhline(linestyle='--')
ax.set_xlabel('$i,j$')
fig.suptitle('Generalized Equipartition Theorem: '+'$\\langle x_i \\partial_{x_j} H\\rangle - I \\text{ with } x_1, x_2, x_3, x_4 = \\varphi, \\varphi_{dc}, \\dot{\\varphi}, \\dot{\\varphi}_{dc}$')



In [ ]:
#this cell is for checking that the initial state is what you want qualitatively, shows phase space histograms
nbins= 30

fig, ax = plt.subplots(2,2, figsize=(10,10))

for ij in [[0, 0], [0, 1], [1, 0], [1, 1]]:
    ax[*ij].hist(initial_state[:,*ij], bins=nbins, density=True)

titles = ['$\\phi$', '$\\dot{\\phi}$', '$\\phi_{dc}$', '$\\dot{\\phi}_{dc}$']
for i, axis in enumerate(ax.ravel()):
    axis.set_title(titles[i])

In [ ]:
#gives a snapshot of the potential at some time in some domain
fig, ax = plt.subplots(1,2, figsize=(12,4))
t_snapshot = 0
fig, _, out = system.show_potential(t_snapshot, manual_domain=[[-4,-4],[4,-1]], figax=[fig,ax[0]], surface=False, show=False)

t_snapshot = system.protocol.t_f-.3
fig, ax[1], out = system.show_potential(system.protocol.t_f-.3, manual_domain=[[-4,-4],[4,-1]], figax=[fig,ax[1]], surface=False, show=False)

# Now we set up the simulation

In [ ]:
# this sets up our simulation to do underdamped langevin dynamics.
# if you want to change the temperature or damping by some amount, you can change the scale factors in this cell
# probably dont want to change anythign else in here though

#NOTE: changing the temperature here will not change the temperature used to generate the EQ distribution,
#NOTE: time is scaled in units of sqrt(LC)

gamma = (lambda_prime/m_prime) * np.sqrt(L*C) / (R*C) 
theta = 1/m_prime
eta = (L/(R**2 * C))**(1/4) * np.sqrt(kT_prime*lambda_prime) / m_prime        
 

damping_scale = 1
temp_scale = 1

gamma = np.multiply(gamma, damping_scale)
eta = np.multiply(eta, np.sqrt(damping_scale*temp_scale))

dynamic = langevin_underdamped.LangevinUnderdamped(theta, gamma, eta, system.get_external_force)
dynamic.mass = system.mass

integrator = rkdeterm_eulerstoch.RKDetermEulerStoch(dynamic)

In [ ]:
#dont change this cell unless you take a look at how the procedures work, this should be fine for most use cases

is_bools = kt.separate_by_state(initial_state[...,0,0])

procedures = [
              sp.ReturnFinalState(),
              sp.MeasureAllState(trial_request=slice(0, 1000), step_request=np.s_[::4]),
              rp.MeasureAllValue(rp.get_dW, 'all_W'),
              rp.MeasureFinalValue(rp.get_dW, 'final_W'),
              sp.MeasureMeanValue(rp.get_kinetic, output_name='kinetic' ),
              sp.MeasureMeanValue(rp.get_potential, output_name='potential'),
              sp.MeasureMeanValue(rp.get_EPT, output_name='equipartition'),
              sp.MeasureMeanValue(rp.get_current_state, output_name = 'zero_means', trial_request=is_bools['0']),
              sp.MeasureMeanValue(rp.get_current_state, output_name = 'one_means', trial_request=is_bools['1']),
              tp.CountJumps(state_slice=np.s_[...,0,0])
             ]

In [ ]:
# here is where you choose the number of steps to simulate and how long to run the sim for.
# note that if your time is longer than the protocol time, the potential will just sit at its final value.

total_time = (system.protocol.t_f-system.protocol.t_i) + 125
dt = 1E-2

nsteps = int(total_time/dt)
nsteps_quick = int(nsteps/10)
sim = simulation.Simulation(integrator.update_state, procedures, nsteps, dt,
                            initial_state)

sim.system = system

# Run the actual sim

In [ ]:
%%time
sim.output = sim.run(verbose=True)

In [ ]:
# Here we assign sim output to some variables for easier access later
# this is a bit of a mess, but it works for now

# this saves trajectory data for the first 1,000 samples. It only saves every 4 time steps to save on memory. this is set in procedures
all_state = sim.output.all_state['states']
# work over time, all trials, all time steps
all_W = sim.output.all_W
# net work at the end per trial
final_W = sim.output.final_W
# the full final state
final_state = sim.output.final_state
# equipartition checks over time, ['values'] is the mean, ['std_error'] is the std error
all_EPT = sim.output.equipartition['values']
# average KE, PE of all particles over time
all_KE = sim.output.kinetic['values']
all_PE = sim.output.potential['values']
# arrays to plot time on the x axis
times = np.linspace(0, total_time, nsteps+1)
# coarse grained version for plotting all_state
all_times = np.linspace(0, total_time, all_state.shape[1])
# info state conditioned averages for each phase space coordinate
z_states = sim.output.zero_means['values']
z_err = np.sqrt(N)*sim.output.zero_means['std_error']
o_states = sim.output.one_means['values']
o_err = np.sqrt(N)*sim.output.one_means['std_error']

# this measures every time the sign of phi changes form + to - 
jumps = sim.output.trajectories


### After running the sim, there are plenty of analysis tools

In [ ]:

fig, ax = plt.subplots()
all_ke  = system.get_kinetic_energy(all_state) / kT_prime

ax.plot(all_times, all_ke.T, alpha=.005);
ax.plot(times, all_KE/kT_prime, zorder=10000, linewidth=1, color='k', label='ensemble average' );
ax.axhline(1, linestyle='--', color='k', alpha=.2, label='kT')
fig.legend()
fig.suptitle('total Kinetic Energy')


In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,5))


ax[0].set_title('Work (kT) vs time')
all_W_kT = all_W/kT_prime
ax[0].plot(times, all_W_kT.T, alpha=.1);
ax[0].plot(times, all_W_kT.mean(axis=0), linewidth=2, color='k', label='ensemble average', alpha=1 );

ax[0].legend()

#this will show you a histogram of the net work, with the mean and +- sigma marked
#note the energy scale is in k_B T
ax[1].set_title('Final Work (kT) histogram')
final_W_kT = final_W/kT_prime
ax[1].hist(final_W_kT, bins=30)
m=(final_W_kT).mean()
s=(final_W_kT).std()
ax[1].axvline(m, color='k', label='mean')
ax[1].axvline(m-3*s, color='k', label=' - 3 sigma')
ax[1].axvline(m+3*s, color='k', label=' + 3 sigma')
ax[1].legend()

In [ ]:
fig, ax = plt.subplots(2,2)

coarse = 25
for i in range(2):
    for j in range(2):
        
        for state, error in zip([z_states, o_states],[z_err, o_err]):
            markers, caps, bars = ax[i,j].errorbar(times[::coarse], state[::coarse,i,j], yerr=error[::coarse,i,j], alpha=.6);
            [bar.set_alpha(.2) for bar in bars]
            for lineval in [ state[0,i,j] + item*error[0,i,j] for item in[-1,1]]:
                ax[i,j].axhline(lineval, linestyle='--', color='k', alpha=.2)


ax[0,0].set_title('$\\phi$')
ax[0,1].set_title('$\\dot{\\phi}$')
ax[1,0].set_title('$\\phi_{dc}$')
ax[1,1].set_title('$\\dot{\\phi}_{dc}$')

fig.suptitle('Info State Average and Std Deviation')


In [ ]:

for item in rc_dict:
    plt.rcParams[item] = rc_dict[item]

# dont need ot look at all of them, this will show every 10th


fig, ax = plt.subplots(2,2, sharex=True)

opacity_values = [.5, .01]
trial_indices = np.s_[::50], np.s_[::1]

for i, [opacity, trials] in enumerate(zip(opacity_values, trial_indices)):
    for j, xy in enumerate([[0,0],[1,0]]):
        ax[i,j].plot(all_times, all_state[trials,:,*xy].T, alpha=opacity);

fig.suptitle('all_state trajectories with t in units of $\\sqrt{LC}$')

ax[0,0].set_ylabel('$\\varphi$')
ax[0,1].set_ylabel('$\\varphi_{dc}$')
ax[1,0].set_ylabel('$\\varphi$')
ax[1,1].set_ylabel('$\\varphi_{dc}$')

tick_params = {'which':'major', 'labelsize':12, 'size':2, 'direction':'inout', 'width':.6}
for item in ax.ravel():
    item.axvline(system.protocol.t_f,c='k', linestyle='dashed')
    item.axvline(1,c='k', linestyle='dashed')
    #item.axvline(5.53,c='k', linestyle='dashed')
    item.grid(True, which='both')
    item.tick_params(**tick_params)


    



In [ ]:
#fig.savefig('all_state_alpha.pdf')

In [ ]:
from FQ_sympy_functions import fidelity
fidelity_dictionary = fidelity(jumps)
print(fidelity_dictionary)

# Below This are more detailed Diagnostic Tools for the protocol and Simulation dt

In [ ]:
from bit_flip_sweep import get_tau_candidate

# this will take a look at this sim and suggest candidate end times for the protocol
# it makes a guess of a better time based only on the phi coordiante
# but its guess is usually wrong because phi_dc matters a lot
tau_candidates, t_crit = get_tau_candidate(sim, burn = int( (1+system.protocol.times[0,1]) / sim.dt) )

In [ ]:
# this cell shows a closeup of the flip part, and all the phase space coordinates
# can try to use it to see if extending, shortening will give better results
fig, ax = plt.subplots(2, figsize=(20,10))

#change plot window
start_idx, end_idx = [ int(item/sim.dt) for item in system.protocol.times[1] ]
duration_idx = int( (end_idx-start_idx)/2)

indices=np.s_[start_idx-duration_idx:end_idx+duration_idx]





for i in range(2):
    for j in range(2):
        
        for state, error in zip([z_states, o_states],[z_err, o_err]):
            markers, caps, bars = ax[i].errorbar(times[indices], state[indices,i,j], yerr=error[indices,i,j], alpha=1, );
            [bar.set_alpha(.2) for bar in bars]


ax[0].set_title('$\\phi$')

ax[1].set_title('$\\phi_{dc}$')


for item in ax:
    item.grid(True, which='both')
    item.axvline(system.protocol.t_f, linewidth=5, color='r',alpha=.4, label='end of protocol')
    item.axvline(tau_candidates[0], color='k', label='range_start')
    item.axvline(tau_candidates[1], color='k', label='range_end',)
    item.axvline(t_crit, linewidth=5, color='g', alpha=.4, label='guessed better time')

ax[1].legend()
fig.suptitle('Ensemble Average of the System Phase Space, conditioned on the initial logical state')

In [ ]:
# looking at energy over time
times = np.linspace(0, total_time, nsteps+1)

fig, ax = plt.subplots(figsize=(15,5));

#potential energy is tricky, because it only makes sense up to a constant-- so i have basically scaled it to look right.
ax.plot(times, (all_PE-all_PE[0])/kT_prime+1, alpha=.8, label='potential');

ax.plot(times, all_KE/kT_prime, alpha=.8, label='kinetic');


ax.legend()
ax.set_title('Avg Energy (k_B T) vs time');
plt.grid(True, which='both')
plt.rc('grid', linestyle="-", color='black')

ax.set_ylim(0,3)

In [ ]:
fig, ax = plt.subplots()
total_E = ( (all_PE-all_PE[0]) + all_KE)/kT_prime +1
ax.plot(times,total_E)
ax.axhline(2, linestyle='dotted')
ax.set_title('Total Energy (k_B T)');

In [ ]:

# plot equipartition check, this is mostly for checking if the distribution is equilibrium

# dont think of these as energies, moreso it measures correlation
EPT = np.abs(all_EPT/kT_prime)

# we convolve over a time window to smooth out the data
import scipy.signal
window_size = int(2.5/dt)
kernel_1d = np.ones(window_size) / float(window_size)
kernel_3d = kernel_1d.reshape(-1, 1, 1) 
EPT = scipy.signal.convolve(EPT, kernel_3d, mode='valid', method='auto')


fig, ax = plt.subplots(1,2, figsize=(15,5));

for i in range(4):
    for j in range(4):
        if i==j:
            ax[0].plot(EPT[:,i,j]-1, alpha=.8);
        else:
            ax[1].plot(EPT[:,i,j], alpha=.8);
 

ax[0].set_title(' \'Equipartition Elements\' (k_B T)  (diagonal elements)');
ax[1].set_title(' \'Equipartition Elements\' (k_B T)  (off-diagonal elements)');

ax[0].legend(['phi', 'phi_dc','v_phi','v_phi_dc'])

#manually set window here
ax[0].set_ylim(-.5,6)
ax[1].set_ylim(-.5,6)
